In [3]:
import cdsapi
import os
from datetime import date, timedelta
from pathlib import Path
import rootpath

ROOT_PATH = Path(rootpath.detect())
os.chdir(str(ROOT_PATH))
sys.path.append(str(ROOT_PATH))

from _shared import getConfig
config = getConfig()

DATA_FOLDER = ROOT_PATH/config['d']['era5']
CLIMATE_FOLDER = DATA_FOLDER/"climate"

START_YEAR = 2012
DELAY_DAYS = 5

RENEW_CURRENT_YEAR = False

c = cdsapi.Client()

In [4]:
# this is necessary unfortunately as a query with a period to close to the current day,
# will fail with an error message
until = date.today() - timedelta(days = DELAY_DAYS)
# until

In [6]:
if (RENEW_CURRENT_YEAR):
    # Remove outdated files
    for f in ['last_year.nc', 'last_month.nc']:
        current_file = CLIMATE_FOLDER/f
        if os.path.exists(current_file):
            os.remove(current_file)

In [7]:
queryBase = {
    'dataset': 'reanalysis-era5-single-levels',
    'options': {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': '2m_temperature',
        'area': [50, 8, 45, 18], # north, west, south, east
        'time': [f"{hour:02d}:00" for hour in range(24)],
        'day': [f"{day:02d}" for day in range(1, (32))],
        'month': [f"{month:02d}" for month in range(1, 13)],
        #'grid': [0.5, 0.5],  # grid in 0.5deg steps in longitude/latitude
    },
}

def download_era5_temperature_years(years, folder):
    for year in years:
        filename = folder/f'{year}.nc'
        if not os.path.exists(filename):
            options = {
                'year': [str(year)],
            }
            c.retrieve(queryBase['dataset'], queryBase['options'] | options, filename)

def download_era5_temperature_last_year(until, folder):
    north, west, south, east = 50.,8, 45, 18
    day, month, year = (until.day, until.month, until.year)

    filename = folder/'last_year.nc' 
    if not os.path.exists(filename):
        options = {
            'month': [f"{month:02d}" for month in range(1, month)],
            'year': [str(year)],
        }
        c.retrieve(queryBase['dataset'], queryBase['options'] | options, filename)

    filename = folder/'last_month.nc' 
    if not os.path.exists(filename):
        options = {
            'day': [f"{day:02d}" for day in range(1, day)],
            'month': [f"{month:02d}"],
            'year': [str(year)],
        }
        c.retrieve(queryBase['dataset'], queryBase['options'] | options, filename)

In [8]:
download_era5_temperature_years(range(START_YEAR, (until.year - 1)), CLIMATE_FOLDER)
print(f'Downloading until {until}')
download_era5_temperature_last_year(until, CLIMATE_FOLDER)

2022-10-19 18:10:02,372 INFO Welcome to the CDS
2022-10-19 18:10:02,373 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-10-19 18:10:02,581 INFO Request is completed
2022-10-19 18:10:02,582 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data8/adaptor.mars.internal-1666184335.4014027-27094-17-f3f0d69b-36ee-4da2-a605-7ec33aa70989.nc to /home/peter/Projects/energy/prep/data/era5/climate/last_year.nc (21.5M)
2022-10-19 18:10:17,520 INFO Download rate 1.4M/s                                                            
2022-10-19 18:10:17,585 INFO Welcome to the CDS
2022-10-19 18:10:17,586 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-10-19 18:10:17,876 INFO Downloading https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data5/adaptor.mars.internal-1666184385.5168219-2408-7-b6f14e50-35e7-4780-b897-b74377bd4ea